In [ ]:
import os
import torch
import torchvision
from torchvision import datasets
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.transforms as tt
import numpy as np
from torchvision.datasets import ImageFolder
import torchvision.models as models

In [ ]:
data_dir_akash = 'dataset_akash'

In [ ]:
train_tfms = tt.Compose([
                         tt.Resize((200, 200)),
                         tt.RandomCrop(200, padding=25, padding_mode='reflect'),
                         tt.RandomHorizontalFlip(),
                         tt.RandomRotation(10),
                         tt.RandomPerspective(distortion_scale=0.2),
                         tt.ToTensor()
                        ])
test_tfms = tt.Compose([tt.Resize((200, 200)),
                        tt.ToTensor()])

In [ ]:
train_set = datasets.ImageFolder(data_dir_akash + '/Train', transform=train_tfms)
test_set = datasets.ImageFolder(data_dir_akash + '/Test', transform=test_tfms)

batch_size_train = 50
batch_size_test = 100
train = torch.utils.data.DataLoader(train_set, batch_size=batch_size_train, shuffle=True, pin_memory=True)
test = torch.utils.data.DataLoader(test_set, batch_size=batch_size_test, shuffle=True, pin_memory=True)

In [ ]:
def train_imshow(input_img):
    classes = ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H',
               'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
              'U', 'V', 'W', 'X', 'Y', 'Z')
    dataiter = iter(input_img)
    images, labels = dataiter.next()
    fig, axes = plt.subplots(figsize=(10, 4), ncols=5)
    for i in range(5):
        ax = axes[i]
        ax.imshow(images[i].permute(1, 2, 0))  
        ax.title.set_text(' '.join('%5s' % classes[labels[i]]))
    plt.show()

In [ ]:
train_imshow(train)

In [ ]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

In [ ]:
device = get_default_device()
device

In [ ]:
train = DeviceDataLoader(train, device)
test = DeviceDataLoader(test, device)

In [ ]:
def accuracy(outputs, labels):
    print("labels", labels)
    print("shape", outputs.shape)
    print("````````````````````````````````````````````````````````````````````````````````````````````````")
    for x in range(0, labels.size(dim=0)):
        print(x)
        print("`")
        print(outputs[x])
        print("`")
        print("max value in array                 ", torch.max(outputs[x]))
        print("index of max value (prediction)    ", torch.argmax(outputs[x]))
        print("true label                         ", labels[x])
        print("`")
        print("`")
        print("````````````````````````````````````````````````````````````````````````````````````````````````")
    
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                 
        loss = F.cross_entropy(out, labels)
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))

In [ ]:
class ASLResnet(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = models.resnet152(pretrained=True)
        num_ftrs = self.network.fc.in_features
        self.network.fc = nn.Linear(num_ftrs, 26)
    
    def forward(self, xb):
        return self.network(xb)
    
    def freeze(self):
        for param in self.network.parameters():
            param.require_grad = False
        for param in self.network.fc.parameters():
            param.require_grad = True
    
    def unfreeze(self):
        for param in self.network.parameters():
            param.require_grad = True

In [ ]:
model = to_device(ASLResnet(), device)

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
        
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            lrs.append(get_lr(optimizer))
            sched.step()
        
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
history = [evaluate(model, test)]
history

In [ ]:
model.freeze()

In [ ]:
epochs = 1
max_lr = 1e-4
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [ ]:
history += fit_one_cycle(epochs, max_lr, model, train, test, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)

In [ ]:
model.unfreeze()

In [ ]:
history += fit_one_cycle(epochs, 1e-5, model, train, test, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)

In [ ]:
torch.save(model.state_dict(), 'resnet152.pt')

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs')

In [ ]:
plot_accuracies(history)

In [ ]:
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs')

In [ ]:
plot_losses(history)

In [ ]:
def plot_lrs(history):
    lrs = np.concatenate([x.get('lrs', []) for x in history])
    plt.plot(lrs)
    plt.xlabel('Batch no.')
    plt.ylabel('Learning rate')
    plt.title('Learning Rate vs. Batch no.')

In [ ]:
plot_lrs(history)

In [ ]:
def predict_image(img, model):
    xb = to_device(img.unsqueeze(0), device)
    yb = model(xb)
    _, preds  = torch.max(yb, dim=1)
    return train_set.classes[preds[0].item()]

In [ ]:
data_dir_rasband = 'dataset_rasband'
test_set_rasband = ImageFolder(data_dir_rasband +'/Test', transform=test_tfms)
test_rasband = torch.utils.data.DataLoader(test_set_rasband, batch_size=10, shuffle=True, pin_memory=True)


In [ ]:
train_imshow(test_rasband)

In [ ]:
test_rasband = DeviceDataLoader(test_rasband, device)

In [ ]:
presaved_model = to_device(ASLResnet(), device)
presaved_model.load_state_dict(torch.load('resnet152.pt', map_location="cuda"))

presaved_model_cpu = ASLResnet()
presaved_model_cpu.load_state_dict(torch.load('resnet152.pt', map_location=torch.device('cpu')))

In [ ]:
evaluate(presaved_model, test_rasband)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
def conf_matrix(model, loader):
    predictions = []
    true_values = []

    for batch_idx, (data, target) in enumerate(loader):
        log_prob = model(data)
        prob = list(torch.exp(log_prob).detach().cpu().clone().numpy())
        y = list(target.detach().cpu().clone().numpy())
        for i in range(len(prob)):
            prob[i] = prob[i].tolist()
            prob[i] = (prob[i]).index(max(prob[i]))
            predictions.append(prob[i])
            true_values.append(y[i])



    cm = confusion_matrix(true_values, predictions)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()

In [ ]:
conf_matrix(presaved_model, test_rasband)